---
title: "Recitation 3"
bibliography: ../reading_list.bib
---

In [ ]:
using LinearAlgebra, Plots, Distributions, Random

# Extending the welfare decomposition from class

In class we studied a decomposition of the gain in utilitarian welfare from moving from a laissez-faire economy to one with perfect insurance (i.e. where everybody receives the same allocation).

Let's now consider a decomposition of the welfare differences between two general policy regimes, where neither necessarily provides perfect insurance. First, some definitions...

## Setup and definitions

Let there be a unit measure of agents in the economy, indexed by $i\in[0,1]$. As before, assume that agents rank probabilistic sequences of consumption according to:

$$ V^{r}_{i} = \mathbb{E}\left[\sum_{t=0}^{T-1}\beta^{t}u(c^{r}_{i,t})\right]$$

where $\{c^{r}_{i,t}\}_{t=0}^{T-1}$ is the sequence of consumption that agent $i$ enjoys under regime $r\in\{A,B\}$.

Let $\bar{c}^{r}_{i}$ be the *certainty equivalent* that solves:
$$ \sum_{t=0}^{T-1}\beta^{t}u(\bar{c}^{r}_{i}) = V^{r}_{i}$$
for $r\in\{A,B\}$. We know that:
$$\sum_{t=0}^{T-1}\beta^{t}u(\bar{c}^{r}_{i}) = \frac{(1-\beta^{T})}{1-\beta}u(\bar{c}^{r}_{i}) = \mathcal{V}(\bar{c}^{r}_{i})$$
Finally, define $C=\int y_{i,t}di = \int c_{i,t}di$ as aggregate or average consumption in the economy. In both regimes we assume that the resource constraint always holds and there is no aggregate uncertainty. Just as in our lecture, we can define the aggregate certainty equivalent as:
$$ \bar{C}^{r} = \int \bar{c}^{r}_{i}di.$$

Note that by definition, we have:
$$ W^{r} = \int V^{r}_{i}di = \int \mathcal{V}(\bar{c}^{r}_{i})di $$

## The decomposition

Note that now we can write:

$$ W^{r} = \mathcal{V}(C^{r}) \times \frac{\mathcal{V}(\bar{C}^{r})}{\mathcal{V}(C^{r})} \times \frac{\int \mathcal{V}(\bar{c}^{r}_{i})di}{\mathcal{V}(\bar{C}^{r})}$$
The first term here is the value obtained if we implemented perfect insurance (everyone gets the same consumption in all periods). The second term is the discount on this perfect insurance when everyone is given the average certainty equivalent, and the third term is the further discount coming from inequality in the allocations of certainty equivalents. You can think of the second and third terms as being "costs" relative to a benchmark. The second term is the cost of imperfect insurance in the model relative to a "perfect insurance" benchmark. The third term is the cost of inequality, stemming from differences in individual certainty equivalent values. Welfare is therefore the product of an aggregate component, an insurance component, and a redistribution component.

Since aggregate income and consumption are unchanged by the policies we consider, the first term cancels out when we decompose the welfare changes when moving from regime $A$ to regime $B$ as:

$$ \frac{W^{B}}{W^{A}} = \frac{\frac{\mathcal{V}(\bar{C}^{B})}{\mathcal{V}(C^{B})}}{\frac{\mathcal{V}(\bar{C}^{A})}{\mathcal{V}(C^{A})}} \times \frac{\frac{\int \mathcal{V}(\bar{c}^{A}_{i})di}{\mathcal{V}(\bar{C}^{A})}}{\frac{\int \mathcal{V}(\bar{c}^{A}_{i})di}{\mathcal{V}(\bar{C}^{A})}}  $$

We can convert these terms into consumption equivalent measures using the formula from [recitation 2](../recitations/recitation-2.qmd):
$$
\begin{eqnarray}
1 + \omega = \left[\frac{W^{B}}{W^{A}}\right]^{1/(1-\sigma)} \\
1+\gamma = \left[\frac{\frac{\mathcal{V}(\bar{C}^{B})}{\mathcal{V}(C^{B})}}{\frac{\mathcal{V}(\bar{C}^{A})}{\mathcal{V}(C^{A})}}\right]^{1/(1-\sigma)} \\
1 + \alpha = \left[\frac{\frac{\int \mathcal{V}(\bar{c}^{A}_{i})di}{\mathcal{V}(\bar{C}^{A})}}{\frac{\int \mathcal{V}(\bar{c}^{A}_{i})di}{\mathcal{V}(\bar{C}^{A})}}\right]^{1/(1-\sigma)}
\end{eqnarray}
$$
with $1+\omega = (1+\gamma)(1+\alpha)$ being the *final* decomposition.

The $\gamma$ term here is the welfare effect of changes in insurance, the $\alpha$ term represents changes in the equality (or inequality) of the distribution of certainty equivalents.

# Adding taxes and transfers to the model

In recitation 2 we wrote down code to solve the life-cycle savings model. Below we'll show how to extend the code to allow for a generic system of taxes and transfers.


In [ ]:
# utility
utility(c,σ) = c^(1 - σ) / (1 - σ)

# tauchen method
function tauchen(N, ρ, σ)
    # Unconditional standard deviation of the process
    σ_y = σ / sqrt(1 - ρ^2)

    # Create the grid
    y_max = 3. * σ_y
    y_min = - 3. * σ_y
    grid = range(y_min, y_max, length=N)

    # Step size
    step = (y_max - y_min) / (N - 1)

    # Initialize transition matrix
    Π = zeros(N, N)

    # Standard normal distribution
    d = Normal(0, 1)

    # Fill in the transition matrix
    for i in 1:N
        for j in 1:N
            if j == 1
                # Probability of transitioning to the lowest state
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ)
            elseif j == N
                # Probability of transitioning to the highest state
                Π[j, i] = 1 - cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            else
                # Probability of transitioning to intermediate states
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ) -
                         cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            end
        end
    end
    return collect(grid), Π
end

# iterate the value function
function iterate_value_function!(p,V,A,t)
    for ai in axes(V,2), yi in axes(V,1)
        v,a_next = solve_value(ai,yi,V,p,t)
        V[yi,ai,t] = v
        A[yi,ai,t] = a_next
    end
end

# solve the model
function solve_model(p)
    (;T,Π,asset_grid) = p
    K_a = length(asset_grid)
    K_y = size(Π,1)
    V = zeros(K_y,K_a,T+1)
    A = zeros(K_y,K_a,T)
    for t in reverse(1:T)
        iterate_value_function!(p,V,A,t)
    end
    return (;V,A)
end

The block above includes everything except the function `solve_value` which we are going to edit slightly:


In [ ]:
function solve_value(ai,yi,V,p,t)
    # note here we assume that p contains a *function*
    # called net_income
    (;σ,β,Π,r,asset_grid,income_grid,net_income) = p
    a = asset_grid[ai]
    y = exp(income_grid[yi,t])
    y_net = net_income(y)
    vmax = -Inf
    amax = 0
    for ai_next in eachindex(asset_grid)
        a_next = asset_grid[ai_next]
        # now we include net income in the budget constraint
        # instead of just income
        c = y_net + a - a_next/(1+r)
        if c>0
            @views v = utility(c,σ) + β*dot(Π[:,yi],V[:,ai_next,t+1])
            if v>vmax
                vmax = v
                amax = a_next
            end
        end
    end
    return vmax,amax
end

Now let's consider a couple of different versions of the net income function.

In [ ]:
no_tax(x) = x

progressive_tax(x,λ,τ) = λ * x^(1-τ)

In [ ]:
μ = log(10.) #
egrid,Π = tauchen(5,0.9,0.1)
T = 60
income_grid = [μ+egrid[i] for i in 1:5, t in 1:T]
p = (;
    T,
    asset_grid = LinRange(0,100,100),
    income_grid,
    β = 0.96,
    r = 0.04,
    Π,
    σ = 2.,
    net_income = no_tax
)

How could we insure that the progressive tax is self financing (i.e. obeys the resource constraint)?  For a given value of $\tau$, we can pick $\lambda$ to insure that the average allocation is equal to average income.

The code below solves for the steady state of $\Pi$ and calculates the $\lambda$ that obeys the resource constraint given $\tau$. Note that this code works under two assumptions:

1. $\mu_{t}$ is constant over time $t$; and
2. The initial draw of $y$ is taken from the stationary distribution of $\Pi$, $\pi^*$.


In [ ]:
function solve_steady_state(Π)
    K = size(Π,2)
    vals,e = eigen(Π)
    return e[:,K] ./ sum(e[:,K])
end

function solve_lambda(τ,ygrid,pi_star)
    return dot(pi_star,ygrid) / dot(pi_star,ygrid.^(1-τ))
end

# choose progressivity
τ = 0.2
# solve for the stationary distribution
pi_star = solve_steady_state(p.Π)
# solve for λ
λ = solve_lambda(τ,exp.(income_grid[:,1]),pi_star)

Now we can try solving the model and comparing values with and without this tax system.


In [ ]:
p_new = (;p...,
    net_income = x->progressive_tax(x,λ,τ))

model0 = solve_model(p)
model1 = solve_model(p_new)

In [ ]:
plot(model0.V[:,1,1],label="no taxes")
plot!(model1.V[:,1,1],label = "progressive taxes")

# Welfare decompositions in the model

To decompose the average welfare gain into components, let's first calculate the terms we need:


In [ ]:
(;σ,β) = p
C = dot(pi_star,exp.(p.income_grid[:,1]))
cbar0 = ((1-σ)*(1-β)/(1-β^T) * model0.V[:,1,1]).^(1/(1-σ))
cbar1 = ((1-σ)*(1-β)/(1-β^T) * model1.V[:,1,1]).^(1/(1-σ))
Cbar0 = dot(pi_star,cbar0)
Cbar1 = dot(pi_star,cbar1)
W0 = dot(pi_star,model0.V[:,1,1])
W1 = dot(pi_star,model1.V[:,1,1])

V_cert(x) = (1-β^T)/(1-β) * utility(x,σ)

And with these in hand we can calculate the decomposition:

In [ ]:
ω = (W1/W0)^(1/(1-σ)) - 1
γ = (V_cert(Cbar1) / V_cert(Cbar0))^(1/(1-σ)) - 1
α = ((W1 / V_cert(Cbar1)) / (W0 / V_cert(Cbar0)))^(1/(1-σ)) - 1

display([1+ω 1+γ 1+α])

Introducing progressive taxation is equivalent to raising consumption by 2.5\% in all periods and states of the world. A decent component (1.8\%) is due to improved insurance against income shocks.

To conclude, let's re-calculate this decomposition for a more persistent value of $\rho$ and interpret the results.


In [ ]:
egrid,Π = tauchen(5,0.99,0.1)
income_grid = [μ+egrid[i] for i in 1:5, t in 1:T]
p = (;
    T,
    asset_grid = LinRange(0,100,100),
    income_grid,
    β = 0.96,
    r = 0.04,
    Π,
    σ = 2.,
    net_income = no_tax
)

# solve for the stationary distribution
pi_star = solve_steady_state(p.Π)
# solve for λ
λ = solve_lambda(τ,exp.(income_grid[:,1]),pi_star)

p_new = (;p...,
    net_income = x->progressive_tax(x,λ,τ))

model0 = solve_model(p)
model1 = solve_model(p_new)

C = dot(pi_star,exp.(p.income_grid[:,1]))
cbar0 = ((1-σ)*(1-β)/(1-β^T) * model0.V[:,1,1]).^(1/(1-σ))
cbar1 = ((1-σ)*(1-β)/(1-β^T) * model1.V[:,1,1]).^(1/(1-σ))
Cbar0 = dot(pi_star,cbar0)
Cbar1 = dot(pi_star,cbar1)
W0 = dot(pi_star,model0.V[:,1,1])
W1 = dot(pi_star,model1.V[:,1,1])

V_cert(x) = (1-β^T)/(1-β) * utility(x,σ)

ω = (W1/W0)^(1/(1-σ)) - 1
γ = (V_cert(Cbar1) / V_cert(Cbar0))^(1/(1-σ)) - 1
α = ((W1 / V_cert(Cbar1)) / (W0 / V_cert(Cbar0)))^(1/(1-σ)) - 1

display([1+ω 1+γ 1+α])

Notice that there are quite large welfare gains, but they are entirely coming from redistribution and not from insurance. When we look at the transition matrix $\Pi$, we see that to numerical precision, the income shocks are essentially permanent:


In [ ]:
Π

which explains why the welfare gains derive purely from redistribution.